<a href="https://colab.research.google.com/github/hansong0219/Advanced-DeepLearning-Study/blob/master/CNN_based_Classification/DenseNet_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DenseNet

DenseNet 은 ResNet 과 다른 방법으로 경사 소실 문제를 해결한다. 숏컷 연결을 사용하는 대신 , 이전 특징 맵 전체가 다음 계층의 입력이 된다. Conv2D 는 크기가 3인 커널을 사용한다. 계층마다 생성되는 특징맵의 개수를 성장률 k라고 할 때, 일반적으로 k = 12 를 사용하지만 , DenseNet 은 그보다 많은 성장률을 사용한다. 따라서 특징 맵의 개수가 k0 일때, 4개 계층으로 구성된 밀집 계층의 끝에서 특징맵의 전체 개수는 4 x k + k0가 된다.

DenseNet도 BN-ReLU-Conv2D 뒤에 밀집 블록을 배치하여, 성장률 k0 = 2 x K 로 특징맵의 개수가 두배가 되기도 한다.

DenseNet은 출력계층에서 Dense() 와 softmax 분류모델 전에 average pooling 을 수행하기를 권장한다. 데이터 확장을 사용하지 않는다면, 드롭아웃 계층이 밀집블록 Conv2D 다음에 와야 한다.

네트워크가 깊어질수록 두 가지의 새로운 문제가 발생하는데, 먼저 모든 계층은 k 개의 특징 맵을 생성하기 때문에 계층 ㅣ에서의 입력 개수는 (l-1) x k + k0 이다. 그에 따라 특징 맵이 깊은 계층에서 급속도로 커지기 때문에 계산 속도가 느려진다. 둘째로 ResNet 과 유사하게 네트워크가 깊어질수록 특징 맵 크기가 줄어들면서 커널의 범위가 늘어난다. 따라서, DenseNet이 병합 연산에서 연결(concatenation)을 사용한다면, 크기의 차이를 일치 시켜야 한다.

이와 같이 특징 맵의 개수가 계산상 비효율 지점까지 증가하는 것을 막기 위해 DenseNet 구조에는 병목 계층이 도입되어 있다. 모든 연결 후에 필터 크기가 4k 인 1 x 1 합성곱이 적용된다. 이 차원 축소 기법은 Conv2D(3) 에서 처리되는 특징 맵의 개수가 빠르게 증가하는 것을 방지한다.

또, 특징 맵 크기가 서로다른 문제를 해결하기 위해 DenseNet 은 심층 신경망을 여러 밀집 블록으로 나누고 전이계층을 통해 서로 연결되게 만들었다. 각 밀집 블록 내에서 특징맵 크기는 일정하다. 전이 계층의 역할은 두 밀집 블록 사이에서 한 특징 맵 크기에서 더 작은 특징 맵 크기로 바꾸는 것이다. 이는 average pooling 계층에 의해서 수행된다. 특징맵이 애버리지 풀링으로 전달되기 전에 Conv2D(1) 을 사용해 그 개수를 특정 압축 비로 줄이는데 보통 이 압축 비는 0.5를 사용한다.

위의 DenseNet 에대한 설계 원칙을 적용시 DenseNet-BC(Bottleneck-Compression) 으로 구성할 수 있다. 여기서 한가지 기억해두어야 할 점은 DenseNet 은 Adam 보다 더 잘 수렴하는 optimizer 인 RMSprop 을 사용한다.

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Input, Flatten, Dropout
from tensorflow.keras.layers import concatenate, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
import os
import numpy as np
import math

In [ ]:
# training parameters
batch_size = 32
epochs = 200
data_augmentation = True

# network parameters
num_classes = 10
num_dense_blocks = 3
use_max_pool = False

# DenseNet-BC with dataset augmentation
# Growth rate   | Depth |  Accuracy (paper)| Accuracy (this)      |
# 12            | 100   |  95.49%          | 93.74%               |
# 24            | 250   |  96.38%          | requires big mem GPU |
# 40            | 190   |  96.54%          | requires big mem GPU |
growth_rate = 12
depth = 100
num_bottleneck_layers = (depth - 4) // (2 * num_dense_blocks)

num_filters_bef_dense_block = 2 * growth_rate
compression_factor = 0.5

In [ ]:
# load the CIFAR10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# input image dimensions
input_shape = x_train.shape[1:]

# mormalize data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    
    print('Learning rate: ', lr)
    
    return lr

In [ ]:
# start model definition
# densenet CNNs (composite function) are made of BN-ReLU-Conv2D
inputs = Input(shape=input_shape)
x = BatchNormalization()(inputs)
x = Activation('relu')(x)
x = Conv2D(num_filters_bef_dense_block,
           kernel_size=3,
           padding='same',
           kernel_initializer='he_normal')(x)
x = concatenate([inputs, x])

# stack of dense blocks bridged by transition layers
for i in range(num_dense_blocks):
    # a dense block is a stack of bottleneck layers
    for j in range(num_bottleneck_layers):
        #Bottlenec Layers
        y = BatchNormalization()(x)
        y = Activation('relu')(y)
        y = Conv2D(4 * growth_rate,
                   kernel_size=1,
                   padding='same',
                   kernel_initializer='he_normal')(y)
        
        if not data_augmentation:
            y = Dropout(0.2)(y)
        
        #Conv2D(3)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Conv2D(growth_rate,
                   kernel_size=3,
                   padding='same',
                   kernel_initializer='he_normal')(y)
        
        if not data_augmentation:
            y = Dropout(0.2)(y)
        x = concatenate([x, y])

    # no transition layer after the last dense block
    if i == num_dense_blocks - 1:
        continue

    # transition layer compresses num of feature maps and reduces the size by 2
    num_filters_bef_dense_block += num_bottleneck_layers * growth_rate
    num_filters_bef_dense_block = int(num_filters_bef_dense_block * compression_factor)
    
    y = BatchNormalization()(x)
    y = Conv2D(num_filters_bef_dense_block,
               kernel_size=1,
               padding='same',
               kernel_initializer='he_normal')(y)
    if not data_augmentation:
        y = Dropout(0.2)(y)
    x = AveragePooling2D()(y)


# add classifier on top
# after average pooling, size of feature map is 1 x 1
x = AveragePooling2D(pool_size=8)(x)
y = Flatten()(x)
outputs = Dense(num_classes,
                kernel_initializer='he_normal',
                activation='softmax')(y)

In [ ]:
# instantiate and compile model
# orig paper uses SGD but RMSprop works better for DenseNet
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(1e-3),
              metrics=['acc'])
model.summary()
plot_model(model, to_file="cifar10-densenet.png", show_shapes=True)

In [ ]:
# run training, with or without data augmentation
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # preprocessing  and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (deg 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(x_train)

    steps_per_epoch = math.ceil(len(x_train) / batch_size)
    # fit the model on the batches generated by datagen.flow().
    model.fit(x=datagen.flow(x_train, y_train, batch_size=batch_size),
              verbose=1,
              epochs=epochs,
              validation_data=(x_test, y_test),
              steps_per_epoch=steps_per_epoch,
              callbacks=callbacks)


    # fit the model on the batches generated by datagen.flow()
    #model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
    ##                    steps_per_epoch=x_train.shape[0] // batch_size,
    #                    validation_data=(x_test, y_test),
    #                    epochs=epochs, verbose=1,
    #                    callbacks=callbacks)

In [ ]:
# score trained model
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])